In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import os
import cv2
import glob
import torch
from Models.LargePNet import *
import tifffile
from Utils.TrainerT16 import *
import matplotlib.pyplot as plt
from Utils.AppendLoad import *
from Utils.DataAug2D import *
import torchvision.models as models 
from Models.LPTISR import *

C:\Users\Dell\.conda\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        vgg19_model = vgg19(pretrained=True)
        self.vgg19_54 = nn.Sequential(*list(vgg19_model.features.children())[:35])

    def forward(self, img):
        return self.vgg19_54(img)

In [3]:
head_dir = r"F:\Opensource\BioTISR\Mito\Training\512data"
Training_GT_path = head_dir + '\\' + r'GT\*.tif'
Training_Raw_path = head_dir + '\\' + r'Noisy\*.tif'
Val_GT_path = head_dir + '\\' + r'ValGT\*.tif'
Val_Raw_path = head_dir + '\\' + r'ValNoisy\*.tif'

X_train = AppendLoad(Training_Raw_path)
y_train = AppendLoad(Training_GT_path)
X_val = AppendLoad(Val_Raw_path)
y_val = AppendLoad(Val_GT_path)

log_dir = head_dir + r'\logfile'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
    
# print("X_train.shape",X_train.shape)
# print("y_train.shape",y_train.shape)
# print("X_val.shape",X_val.shape)
# print("y_val.shape",y_val.shape)
# 
# train_data = Data.TensorDataset(X_train,y_train)
# val_data = Data.TensorDataset(X_val,y_val)

100%|██████████| 24/24 [00:20<00:00,  1.19it/s]


In [4]:
print(X_train.shape)

torch.Size([480, 1, 7, 512, 512])


In [5]:
X_train = X_train.squeeze(1)
y_train = y_train.squeeze(1)
X_val = X_val.squeeze(1)
y_val = y_val.squeeze(1)
train_data = Data.TensorDataset(X_train,y_train)
val_data = Data.TensorDataset(X_val,y_val)

In [6]:
# 定义一个类来存储参数  
class Options:  
    def __init__(self):  
        self.LR = 0.001
        self.batchsize = 1 
        self.epoch_num = 50 
        self.MSE_weight = 1
        self.MAE_weight = 0.05 
        self.SSIM_weight = 0 
        self.epoch_critic = 10
        self.useinit = 0
        self.ModelType = 1   
        self.StepLR = 1
        self.ExpoLR = 0
        self.val = 1
        self.use_dir =0
        self.use_norm = 1
        self.val_data_dir = []
        self.innerpoint = []
        self.instanceimage = r''
opt = Options()

In [7]:
model = LPTISR(2,7,False)
opt.LR = 0.001
save_path = head_dir+r'\logfile\LPTISR'
TrainerT16(model, train_data, val_data, save_path, opt)

drop path: Identity()
drop path: Identity()
drop path: Identity()


C:\Users\Dell\.conda\envs\DL\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


0


Training epoch:   0%|          | 0/480 [00:00<?, ?it/s]C:\Users\Dell\.conda\envs\DL\lib\site-packages\torch\nn\functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
Training epoch: 100%|██████████| 480/480 [19:46<00:00,  2.47s/it, Epoch: 1, Loss: 0.005364761125525721] 


Best ValPSNR: 24.062500
Epoch: 0, Train Loss: 0.040516, Train MSE: 0.032768, Train MAE: 0.154958
Epoch: 0, Val Loss: 0.004768, Val MSE: 0.004768, Val MAE: 0.042267, Val SSIM: 0.000000
1


Training epoch: 100%|██████████| 480/480 [10:16<00:00,  1.28s/it, Epoch: 2, Loss: 0.004717243171587701] 


Best ValPSNR: 25.750000
Epoch: 1, Train Loss: 0.007835, Train MSE: 0.004774, Train MAE: 0.061204
Epoch: 1, Val Loss: 0.003054, Val MSE: 0.003054, Val MAE: 0.035980, Val SSIM: 0.000000
2


Training epoch:  22%|██▏       | 106/480 [06:16<22:09,  3.55s/it, Epoch: 3, Loss: 0.0033074796104300185]


KeyboardInterrupt: 